In [ ]:
import os

import numpy as np
import pandas as pd

import xarray as xr

import xesmf as xe

In [1]:
path_d = "../results/"
path_r = "../temp/NSRDB/"
names = [ "NSRDB_4km.nc", "NSRDB_2km.nc" ]
with xr.open_dataset(path_d + names[0]) as ds_4:
    with xr.open_dataset(path_d + names[1]) as ds_2:
        #ds_4 = ds_4.sel(ds_4["time"].dt.year.isin(
        #    np.unique( ds_2["time"].dt.year.values ) ))
        ds_4["Temperature"] += 273.15
        ds_4["Dew Point"] += 273.15
        ds_2["Temperature"] += 273.15
        ds_2["Dew Point"] += 273.15
        ds_2 = ds_2.drop_vars("Cloud Type")#

        # Cambiamos la resolución.
        regridder = xe.Regridder( ds_4, ds_2, "conservative" )
        ds_2_int = regridder( ds_4, keep_attrs = True )

        v_uv = [ "Global Horizontal UV Irradiance (280-400nm)",
            "Global Horizontal UV Irradiance (295-385nm)" ]

        f_int = ds_2 / ds_2_int.drop_vars( v_uv )
        f_int = f_int.where( f_int.apply(np.isfinite) & ~f_int.isnull(), 1 )
        f_int = f_int.mean( "time", skipna = True )
        f_int[ v_uv[0] ] = f_int["GHI"]
        f_int[ v_uv[1] ] = f_int["GHI"]
        f_int.to_netcdf(path_d + "NSRDB_f_int.nc")

        ds_2_corr = ds_2_int * f_int
        ds_2_corr["UVHI"] = ds_2_corr[ v_uv[0] ] + ds_2_corr[ v_uv[1] ]
        ds_2_corr = ds_2_corr.rename_vars( {
            "Relative Humidity": "Relative_Humidity",
            "Wind Speed": "Wind_Speed", "Wind Direction": "Wind_Direction",
            "Dew Point": "Dew_Point", "Surface Albedo": "Surface_Albedo",
            "Precipitable Water": "Precipitable_Water",
            "Solar Zenith Angle": "Solar_Zenith_Angle",
            v_uv[0]: "UV_A", v_uv[1]: "UV_B" } )
        
        f_regrid = "../temp/WRF_miroc_1985_2014_00054.nc"
        ds_rg = xr.open_dataset(f_regrid).drop_dims("bnds")
        ds_rg = ds_rg.isel( { "south_north": slice(1, 4),
            "west_east": slice(2, 5) } )
        ds_rg_2 = ds_rg.copy()
        ds_rg["XLAT"] = ds_rg["XLAT"].isel({"west_east": 0})
        ds_rg["XLONG"] = ds_rg["XLONG"].isel({"south_north": 0})
        ds_rg = ds_rg.rename_vars( { "XLAT": "lat",
            "XLONG": "lon", "XTIME": "time" } )
        ds_rg = ds_rg.swap_dims( { "south_north": "lat", 
            "west_east": "lon", "XTIME": "time" } )
        
        # Cambiamos la resolución.
        regridder = xe.Regridder( ds_2_corr, ds_rg, "conservative" )
        ds_2_rg = regridder( ds_2_corr, keep_attrs = True )

        ds_f = ds_rg_2.reindex( { "XTIME": ds_2_rg["time"].values }
            ).drop_vars("Dew_Point")
        dims = ["XTIME", "south_north", "west_east"]

        vars = list(ds_2_rg.keys())

        for v in vars: ds_f[v] = ( dims, ds_2_rg[v].values )
        ds_f[[v]].to_netcdf( path_r + "quantile_prep/NSRDB_int_001.nc" ) 

        for i in ["UV_A", "UV_B", "Solar_Zenith_Angle", "Dew_Point", "DHI"]:
            vars.remove(i)

        for v in vars:
            if not os.path.exists( path_r + "quantile_vars/" + v + "/" ):
                os.mkdir( path_r + "quantile_vars/" + v + "/" )
            ds_f[[v]].to_netcdf( path_r + "quantile_vars/"
                + v + "/WRF_miroc_1985_2014_00054.nc" ) 